# Tilføje EPS fra fremtiden - og omdøbe kolonne navne

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

scaled_df_fama = pd.read_csv("/content/drive/MyDrive/Cand.fælles/Speciale/data_prep_2Q_est.csv")

In [ ]:
scaled_df_fama

In [ ]:
#Henter actuals ind
actuals = pd.read_csv("/content/drive/MyDrive/Cand.fælles/Speciale/actuals_for_2Q_est.csv")

In [ ]:
actuals["Quarter_end_date"]= pd.to_datetime(actuals["Quarter_end_date"])
actuals["Period End Date"]= pd.to_datetime(actuals["Period End Date"])

In [ ]:
actuals.dtypes

In [ ]:
from pandas.tseries.offsets import MonthEnd
actuals["forecast_period_date"] = actuals["Quarter_end_date"] - MonthEnd(12)


In [ ]:
actuals

In [ ]:
scaled_df_fama.dtypes

In [ ]:
scaled_df_fama["Calc Date"]= pd.to_datetime(scaled_df_fama["Calc Date"])

In [ ]:
scaled_df_fama[scaled_df_fama.Instrument=="020813101"].head(5)

In [ ]:
data_output = actuals.merge(scaled_df_fama, how="inner", left_on=["forecast_period_date", "Instrument"], right_on=["Calc Date", "Instrument"])

In [ ]:
data_output

# EPS Acutal - X = Den værdi vi ønsker at forecaste
# EPS Actual - Y = Den senest tilgængelige EPS Værdi 

In [ ]:
drop_list= ["Instrument","CFACSHR", "EndOfMonth","FY","FY_end","Fiscal_year_quarter","HQ_country_code(loc)","company_status","consol","CUSIP","data_year_quarter","count_obs","currency","datafmt","fqtr",
"index","industry_code(sic)","native_currency","popsrc","reporting_frq","ticker","variable","forecast_period_date","Calc Date_x","Quarter_end_date_x","Period End Date"]

In [ ]:
data_output.drop(drop_list,axis=1,inplace=True)

In [ ]:
data_output.info()

In [ ]:
drop_list = ["Period End Date_y","Period End Date_x","Report_Date_Earnings(Rdq)","date_x","date_y","indfmt","perm","value","date_format"]

In [ ]:
data_output.drop(drop_list,axis=1,inplace=True)

In [ ]:
rename_columns = {"Earnings Per Share Reported - Actual_x" : "EPS_actual", "Earnings Per Share Reported - Actual_y": "EPS_lagged","Quarter_end_date_y": "periode_regnskabstal", "Calc Date_y": "Forecasttidspunkt"}
data_output.rename(columns=rename_columns,inplace=True)

In [ ]:
data_output.info()

In [ ]:
data_output.EPS_actual.idxmax()

In [ ]:
data_output.iloc[91238]

In [ ]:
data_output = data_output[data_output.EPS_actual.between(data_output.EPS_actual.quantile(0.01),data_output.EPS_actual.quantile(0.99))]

In [ ]:
data_output.drop_duplicates(subset=["Forecasttidspunkt","GVKEY"],inplace=True)

In [ ]:
data_output

In [ ]:
data_output.drop(["Unnamed: 0_x","Unnamed: 0_y", "analyst_EPS_mean", "analyst_high", "analyst_low", "analyst_std_mean" ], axis=1, inplace=True)

In [ ]:
analyst_Q4 = pd.read_csv("/content/drive/MyDrive/Cand.fælles/Speciale/Analyst data/analyst_Q4_longrun_falseindex.csv")
analyst_Q4

In [ ]:
analyst_Q4.dtypes

In [ ]:
analyst_Q4["Calc Date"]= pd.to_datetime(analyst_Q4["Calc Date"])

In [ ]:
analyst_Q4

In [ ]:
analyst_Q4.notna().sum()

In [ ]:
data_output.info()

In [ ]:
data_output.head()

In [ ]:
data_output[data_output.cusip=="037833100"]

In [ ]:
data_output = data_output.merge(analyst_Q4, how="left", left_on=["Forecasttidspunkt", "cusip"], right_on=["Calc Date", "Instrument"])

In [ ]:
data_output

In [ ]:
data_output.drop(["Calc Date", "Period End Date"], axis=1, inplace = True)

In [ ]:
# Aktiedata ind igen
aktiedata_altprice = pd.read_csv("/content/drive/MyDrive/Cand.fælles/Speciale/aktie_data_for_merging.csv")

aktiedata_altprice = aktiedata_altprice[["date","PERMNO","ALTPRC"]]

aktiedata_altprice["date"] = pd.to_datetime(aktiedata_altprice["date"])

from pandas.tseries.offsets import MonthEnd
aktiedata_altprice["slutmåned"] = aktiedata_altprice["date"] - MonthEnd(0)

aktiedata_altprice

data_output = data_output.merge(aktiedata_altprice,how="left",left_on=["Forecasttidspunkt","PERMNO"],right_on=["slutmåned","PERMNO"])

In [ ]:
data_output

In [ ]:
data_output.info()

In [ ]:
data_output[data_output["Earnings Per Share Reported - Mean"].notna()==True].Forecasttidspunkt.dt.year.value_counts().sort_index()

In [ ]:
data_output.rename(columns={"Earnings Per Share Reported - Mean": "analyst_EPS_mean", "Earnings Per Share Reported - Standard Deviation": "analyst_std_mean", "Earnings Per Share Reported - High": "analyst_high", "Earnings Per Share Reported - Low": "analyst_low" }, inplace=True)

In [ ]:
data_output

In [ ]:
data_output.to_csv("/content/drive/MyDrive/Cand.fælles/Speciale/datarobot_input/input_4Q.csv.zip",index=False)